In [43]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import csv

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [44]:
!wget http://cocl.us/Geospatial_data
print('Data downloaded!')

--2019-02-17 07:12:06--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-02-17 07:12:06--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-17 07:12:06--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-17 07:12:06--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [51]:
with open('Geospatial_data.3') as csv_file:
    toronto_data = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in toronto_data:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]} has latitude {row[1]} , has longitude {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')

                  
toronto_data


Column names are Postal Code, Latitude, Longitude
	M1B has latitude 43.8066863 , has longitude -79.1943534.
	M1C has latitude 43.7845351 , has longitude -79.1604971.
	M1E has latitude 43.7635726 , has longitude -79.1887115.
	M1G has latitude 43.7709921 , has longitude -79.2169174.
	M1H has latitude 43.773136 , has longitude -79.2394761.
	M1J has latitude 43.7447342 , has longitude -79.2394761.
	M1K has latitude 43.7279292 , has longitude -79.2620294.
	M1L has latitude 43.7111117 , has longitude -79.2845772.
	M1M has latitude 43.716316 , has longitude -79.2394761.
	M1N has latitude 43.692657 , has longitude -79.2648481.
	M1P has latitude 43.7574096 , has longitude -79.273304.
	M1R has latitude 43.7500715 , has longitude -79.2958491.
	M1S has latitude 43.7942003 , has longitude -79.2620294.
	M1T has latitude 43.7816375 , has longitude -79.3043021.
	M1V has latitude 43.8152522 , has longitude -79.2845772.
	M1W has latitude 43.7995252 , has longitude -79.3183887.
	M1X has latitude 43.83612

In [47]:
with open('Geospatial_data.2') as csv_file:
    toronto_data = csv.reader(csv_file, delimiter=',')
toronto_data


In [52]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbourhoods = pd.DataFrame(columns=column_names)
neighbourhoods

,Borough,Neighborhood,Latitude,Longitude


In [53]:
import pandas as pd
data = {'Postcode': ['M3A', 'M4A', 'M5A' ,'M6A', 'M7A', 'M9A', 'M1B', 'M3B','M4B', 'M5B','M6B', 'M9B','M1C','M3C','M4C','M5C', 'M6C', 'M9C', 'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G','M5G','M6G', 'M1H', 'M2H', 'M3H', 'M4H', 'M5H','M6H' ,'M1J', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J','M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L', 'M2L', 'M3L', 'M4L', 'M5L','M6L' ,'M9L', 'M1M', 'M2M', 'M3M', 'M4M',  'M5M', 'M6M',  'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M9N','M1P', 'M2P', 'M4P', 'M5P', 'M6P','M9P', 'M1R', 'M2R','M4R','M5R','M6R','M7R', 'M9R','M1S', 'M4S','M5S','M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V', 'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X', 'M4Y', 'M7Y', 'M8Y', 'M8Z'],'Borough':['North York', 'North York', 'Downtown Toronto','North York', 'Queens Park', 'Etobicoke', 'Scarborough' , 'North York', 'East York', 'Downtown Toronto', 'North York','Etobicoke','Scarborough', 'North York', 'East York', 'Downtown Toronto','York','Etobicoke','Scarborough' ,'East Toronto','Downtown Toronto', 'York','Scarborough', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'Scarborough', 'North York', 'North York','East York', 'Downtown Toronto','West Toronto', 'Scarborough', 'North York', 'North York','East York', 'Downtown Toronto','West Toronto', 'Scarborough', 'North York', 'North York','East Toronto','Downtown Toronto', 'West Toronto', 'Scarborough','North York', 'North York','East Toronto', 'Downtown Toronto', 'North York', 'North York', 'Scarborough', 'North York','North York','East Toronto','North York', 'York','North York','Scarborough', 'North York', 'North York', 'Central Toronto','Central Toronto', 'York', 'York','Scarborough','North York', 'Central Toronto','Central Toronto','West Toronto','Etobicoke','Scarborough','North York','Central Toronto','Central Toronto','West Toronto','Mississaug', 'Etobicoke','Scarborough','Central Toronto','Downtown Toronto','West Toronto','Scarborough','Central Toronto','Downtown Toronto','Scarborough','Central Toronto','Downtown Toronto','Etobicoke','Etobicoke','Scarborough','Downtown Toronto','Downtown Toronto','Etobicoke','Etobicoke','Scarborough','Downtown Toronto','Downtown Toronto','Etobicoke','Downtown Toronto','East Toronto','Etobicoke','Etobicoke'],'Neighbourhood':['Parkwoods','Victoria Village', "Harbourfront,Regent Park", "Lawrence Heights,Lawrence Manor",'Queens Park','Islington Avenue',"Rouge,Malvern" ,'Don Mills North',"Woodbine Gardens,Parkview Hill","Ryerson,Garden District",'Glencairn',"Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park","Highland Creek,Rouge Hill,Port Union","Flemingdon Park,Don Mills South",'Woodbine Heights','St. James Town','Humewood-Cedarvale',"Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe","Guildwood,Morningside,West Hill",'The Beaches','Berczy Park','Caledonia-Fairbanks','Woburn','Leaside','Central Bay Street','Christie','Cedarbrae','Hillcrest Village',"Bathurst Manor,Downsview North,Wilson Heights",'Thorncliffe Park',"Adelaide, King,Richmond","Dovercourt Village,Dufferin",'Scarborough Village',"Fairview,Henry Farm,Oriole","Northwood Park,York University",'East Toronto',"Harbourfront East,Toronto Islands,Union Station","Little Portugal,Trinity","East Birchmount Park,Ionview,Kennedy Park",'Bayview Village',"CFB Toronto,Downsview East","The Danforth West,Riverdale","Design Exchange,Toronto Dominion Centre","Brockton,Exhibition Place,Parkdale Village","Clairlea,Golden Mile,Oakridge","Silver Hills,York Mills",'Downsview West',"The Beaches West,India Bazaar","Commerce Court, Victoria Hotel", "Maple Leaf Park , North Park , Upwood Park ", 'Humber Summit', "Cliffcrest,Cliffside , Scarborough Village West", "Newtonbrook ,  Willowdale" , 'Downsview Central', 'Studio District', "Bedford Park,Lawrence Manor East","Del Ray,Keelesdale,Mount Dennis,Silverthorn","Emery,Humberlea","Birch Cliff,Cliffside West",'Willowdale South','Downsview Northwest','Lawrence Park','Roselawn',"The Junction North,Runnymede",'Weston',"Dorset Park,Scarborough Town Centre,Wexford Heights",'York Mills West','Davisville North',"Forest Hill North,Forest Hill West","High Park,The Junction South",'Westmount'
,"Maryvale,Wexford",'Willowdale West','North Toronto West',"The Annex,North Midtown,Yorkville","Parkdale,Roncesvalles",'Canada Post Gateway Processing Centre',"Kingsview Village,Martin Grove Gardens,Richview Gardens, St. Phillips",'Agincourt','Davisville',"Harbord,University of Toronto","Runnymede,Swansea","Clarks Corners,Sullivan,Tam O Shanter","Moore Park,Summerhill East","Chinatown,Grange Park,Kensington Market","Agincourt North,L Amoreaux East,Milliken,Steeles East","Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West","CN Tower,Bathurst Quay, Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara","Humber Bay Shores,Mimico South,New Toronto","Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown","L Amoreaux West,Steeles West",'Rosedale','Stn A PO Boxes 25 The Esplanade',"Alderwood,Long Branch",'Northwest','Upper Rouge',"Cabbagetown,St. James Town","First Canadian Place,Underground city","The Kingsway,Montgomery Road,Old Mill North",'Church and Wellesley','Business Reply Mail Processing Centre 969 Eastern',"Humber Bay,Kings Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea","Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor"], 'Latitude':[43.8066863,43.7845351,43.7635726,43.7709921,43.773136,43.7447342,43.7279292,43.7111117,43.716316,43.692657,43.7574096,43.7500715,43.7942003,43.7816375,43.8152522,43.7995252,43.8361247,43.8037622,43.7785175,43.7869473,43.7574902,43.789053,43.7701199,43.7527583,43.7827364,43.7532586,43.7459058,43.7258997,43.7543283,43.7679803,43.7374732,43.7390146,43.7284964,43.7616313,43.7258823,43.7063972,43.6953439,43.6763574,43.7090604,43.7053689,43.685347,43.6795571,43.6689985,43.6595255,43.7280205,43.7127511,43.7153834,43.7043244,43.6895743,43.6864123,43.6795626,43.667967,43.6658599,43.6542599,43.6571618,43.6514939,43.6447708,43.6579524,43.6505712,43.6408157,43.6471768,43.6481985,43.7332825,43.7116948,43.6969476,43.6727097,43.6626956,43.6532057,43.6289467,43.6464352,43.6484292,43.718518,43.709577,43.6937813,43.6890256,43.669542,43.6690051,43.6479267,43.6368472,43.7137562,43.6911158,43.6731853,43.6616083,43.6489597,43.6515706,43.6623015,43.6369656,43.6627439,43.6056466,43.6024137,43.6536536,43.6362579,43.6288408,43.6678556,43.6509432,43.6435152,43.7563033,43.7247659,43.706876,43.696319,43.6889054,43.7394164,43.7067483], 'Longitude':[-79.1943534,-79.1604971,-79.1887115,-79.2169174,-79.2394761,-79.2394761,-79.2620294,-79.2845772,-79.2394761,-79.2648481,-79.273304,-79.2958491,-79.2620294,-79.3043021,-79.2845772,-79.3183887,-79.2056361,-79.3634517,-79.3465557,-79.385975,-79.3747141,-79.4084928,-79.4084928,-79.4000493,-79.4422593,-79.3296565,-79.352188,-79.340923,-79.4422593,-79.4872619,-79.4647633,-79.5069436,-79.4956974,-79.5209994,-79.3155716,-79.309937,-79.3183887,-79.2930312,-79.3634517,-79.3493719,-79.3381065,-79.352188,-79.3155716,-79.340923,-79.3887901,-79.3901975,-79.4056784,-79.3887901,-79.3831599,-79.4000493,-79.3775294,-79.3676753,-79.3831599,-79.3606359,-79.3789371,-79.3754179,-79.3733064,-79.3873826,-79.3845675,-79.3817523,-79.3815764,-79.3798169,-79.4197497,-79.4169356,-79.4113072,-79.4056784,-79.4000493,-79.4000493,-79.3944199,-79.374846,-79.3822802,-79.4647633,-79.4450726,-79.4281914,-79.453512,-79.4225637,-79.4422593,-79.4197497,-79.4281914,-79.4900738,-79.4760133,-79.4872619,-79.4647633,-79.456325,-79.4844499,-79.3894938,-79.615819,-79.321558,-79.5013207,-79.5434841,-79.5069436,-79.4985091,-79.5209994,-79.5322424,-79.5547244,-79.5772008,-79.5659633,-79.5322424,79.5181884,-79.5322424,-79.5547244,-79.5884369,-79.5940544
]}
df = pd.DataFrame(data)
print (df)

    Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A       Queens Park   
5        M9A         Etobicoke   
6        M1B       Scarborough   
7        M3B        North York   
8        M4B         East York   
9        M5B  Downtown Toronto   
10       M6B        North York   
11       M9B         Etobicoke   
12       M1C       Scarborough   
13       M3C        North York   
14       M4C         East York   
15       M5C  Downtown Toronto   
16       M6C              York   
17       M9C         Etobicoke   
18       M1E       Scarborough   
19       M4E      East Toronto   
20       M5E  Downtown Toronto   
21       M6E              York   
22       M1G       Scarborough   
23       M4G         East York   
24       M5G  Downtown Toronto   
25       M6G  Downtown Toronto   
26       M1H       Scarborough   
27       M2H        North York   
28       M3H  